# Etsy Dataset

The cleaning steps in this notebook are motivated by the following planned analysis questions:
   - Most popular occasions
   - Median price of products (non-wedding products vs. wedding products)
   - Most frequent product type (single-stems or bouquets)
   - Most frequent product type (non-wedding products vs. wedding products)
   - Most and least popular tags
   - Distribution of quantities (one-of-a-kind vs. multiples)
   - Percentage of made-to-order products vs. pre-made
   - Most and least popular intended recipients
   - Most and least popular occasions
   - Most popular styles

## Load

In [1]:
# load all data into a data frame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  


In [2]:
# Load Data
full_dataset = "/Users/kristinafrazier/documents/projects/etsy/data/nov_2020/all_etsy_listings_raw.csv"

# Preliminary cleaning from previous analysis
converters = {"tags": lambda x: x.strip("[]").replace("'","").split(", "),"materials": lambda x: x.strip("[]").replace("'","").split(", ")}
data = pd.read_csv(full_dataset, converters = converters)

df = pd.DataFrame(data)

## Assess

In [3]:
df.head()

,Unnamed: 0,listing_id,state,user_id,category_id,title,description,creation_tsz,ending_tsz,original_creation_tsz,...,is_customizable,is_digital,file_data,should_auto_renew,language,has_variations,taxonomy_id,taxonomy_path,used_manufacturer,is_vintage
0,0,742705844,active,89559032,NaN,"Poppy Flower, 12 Peach Crepe Paper Poppies, Pa...",Important Holiday Deadlines\nAs of today (11/2...,1606608041,1616972441,1574724686,...,True,False,NaN,True,en-US,False,1299,"['Paper & Party Supplies', 'Paper', 'Origami',...",False,False
1,1,707163831,active,5945447,NaN,Downloadable Crepe Paper Orchid Template/PDF P...,Make your own crepe paper orchids using this d...,1606607709,1616972109,1558137436,...,False,True,2 PDF,False,en-US,False,1299,"['Paper & Party Supplies', 'Paper', 'Origami',...",False,False
2,2,911675611,active,32178173,NaN,5 Red Mulberry Paper Ribbons pressed in rolls ...,Saa Mulberry Paper Ribbons \n\nContains: 5 rol...,1606605751,1616970151,1606311789,...,True,False,NaN,False,en-US,False,6606,"['Craft Supplies & Tools', 'Party & Gifting', ...",False,False
3,3,60079349,active,21841,NaN,Handmade Crepe Grass Green Fringe,"Made from grass green crepe paper, this ultra-...",1606604287,1616968687,1288183624,...,True,False,NaN,False,en-US,False,562,['Craft Supplies & Tools'],False,False
4,4,161814267,active,21841,NaN,Handmade Crepe Fringe - Navy Blue,"Made from navy blue crepe paper, this fringe t...",1606604239,1616968639,1378423060,...,True,False,NaN,False,en-US,False,562,['Craft Supplies & Tools'],False,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5300 entries, 0 to 5299
Data columns (total 50 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             5300 non-null   int64  
 1   listing_id             5300 non-null   int64  
 2   state                  5300 non-null   object 
 3   user_id                5300 non-null   int64  
 4   category_id            0 non-null      float64
 5   title                  5300 non-null   object 
 6   description            5300 non-null   object 
 7   creation_tsz           5300 non-null   int64  
 8   ending_tsz             5300 non-null   int64  
 9   original_creation_tsz  5300 non-null   int64  
 10  last_modified_tsz      5300 non-null   int64  
 11  price                  5300 non-null   float64
 12  currency_code          5300 non-null   object 
 13  quantity               5300 non-null   int64  
 14  sku                    5300 non-null   object 
 15  tags

- **'Unnamed: 0 ' is the unique_id**
- **Duplicate records**
- **Non-USD records**

In [5]:
df.columns

Index(['Unnamed: 0', 'listing_id', 'state', 'user_id', 'category_id', 'title',
       'description', 'creation_tsz', 'ending_tsz', 'original_creation_tsz',
       'last_modified_tsz', 'price', 'currency_code', 'quantity', 'sku',
       'tags', 'materials', 'shop_section_id', 'featured_rank', 'state_tsz',
       'url', 'views', 'num_favorers', 'shipping_template_id',
       'processing_min', 'processing_max', 'who_made', 'is_supply',
       'when_made', 'item_weight', 'item_weight_unit', 'item_length',
       'item_width', 'item_height', 'item_dimensions_unit', 'is_private',
       'recipient', 'occasion', 'style', 'non_taxable', 'is_customizable',
       'is_digital', 'file_data', 'should_auto_renew', 'language',
       'has_variations', 'taxonomy_id', 'taxonomy_path', 'used_manufacturer',
       'is_vintage'],
      dtype='object')

In [6]:
# Cycle through all columns to inspect of they are relevant to the analysis
df['is_vintage'].value_counts()

False    4815
True      485
Name: is_vintage, dtype: int64

In [7]:
df[df.is_customizable == True].loc[:,['title','description','when_made']]

,title,description,when_made
0,"Poppy Flower, 12 Peach Crepe Paper Poppies, Pa...",Important Holiday Deadlines\nAs of today (11/2...,made_to_order
2,5 Red Mulberry Paper Ribbons pressed in rolls ...,Saa Mulberry Paper Ribbons \n\nContains: 5 rol...,2020_2020
3,Handmade Crepe Grass Green Fringe,"Made from grass green crepe paper, this ultra-...",2020_2020
4,Handmade Crepe Fringe - Navy Blue,"Made from navy blue crepe paper, this fringe t...",made_to_order
13,"Paper Flowers Wall, Nursery Wall Decor, Nurser...",Paper flowers for nursery wall decor will turn...,made_to_order
...,...,...,...
5278,Red giant paper self-standing rose with stem &...,This wonderful rose is absolutely on point: go...,made_to_order
5279,Handmade Crepe Fringe -- Shiny Gold,Made from the most awesome shiny gold crepe pa...,made_to_order
5291,Art Deco Valentine with Puffy Heart,This is an excellent example of Nouveau style....,1910s
5293,Giant Paper Flower with Stem | Huge Paper Peon...,"It would be perfect, stunning paper flower wil...",made_to_order


- **Irrelevant columns:**
    - "active" (these are all active listings)
    - "category_id" (all null data)
    - "creation_tsz" (
    - "ending_tsz"
    - "original_creation_tsz"
    - "last_modified_tsz"
    - "sku"
    - "shop_section_id"
    - "featured_rank"
    - "state_tsz"
    - "shipping_template_id"
    - "processing_min"
    - "processing_max"
    - "is_supply"
    - "item_weight"
    - "item_length"
    - "item_width"
    - "item_height"
    - "item_dimensions_unit"
    - "is_private"
    - "non_taxable"
    - "is_customizable"
    - "is_digital"
    - "file_data"
    - "should_auto_renew"
    - "language"
    - "has_variations"
    - "taxonomy_id"
    - "used_manufacturer"

In [ ]:
df["tags"].value_counts()

In [ ]:
df['materials'].value_counts()

In [ ]:
df['style'].value_counts()

- **The "tags","materials", and "style" columns contain lists and should be separate tables.**

In [ ]:
df.describe()

In [ ]:
df[df.price == df.price.max()][['price','title','description','url']]

In [ ]:
df[df.price == df.price.max()]

In [ ]:
df[df.price == df.price.max()]['url']

In [ ]:
df[df.currency_code == 'USD'].describe()

In [ ]:
df_usd = df[(df.currency_code == 'USD')]
df_usd[df_usd.price == df_usd.price.max()]['url']

In [ ]:
df_usd.sort_values('price')['price'].tail(10)

In [ ]:
plt.figure(figsize = [20,5])

bins = np.arange(0,5000,25)

plt.subplot(1,3,1)
plt.hist(x = df_usd.price, bins = bins)

plt.subplot(1,3,2)
plt.hist(x = df_usd.price, bins = bins)
plt.xlim(1,2000)

plt.subplot(1,3,3)
plt.hist(x = df_usd.price, bins = bins)
plt.xlim(1,1000)
         
;

The price data is right skewed but not strongly enough to justify excluding the $4,999 listing from the dataset. It will just justify taking median measures of prices instead of averages.

## Plan

### Cleaning Tasks:

1. 'Unnamed: 0 ' is the unique_id
2. Duplicate records
3. Non-USD records
4. Irrelevant columns:
    - "active" (these are all active listings)
    - "category_id" (all null data)
    - "creation_tsz" (
    - "ending_tsz"
    - "original_creation_tsz"
    - "last_modified_tsz"
    - "sku"
    - "shop_section_id"
    - "featured_rank"
    - "state_tsz"
    - "shipping_template_id"
    - "processing_min"
    - "processing_max"
    - "is_supply"
    - "item_weight"
    - "item_length"
    - "item_width"
    - "item_height"
    - "item_dimensions_unit"
    - "is_private"
    - "non_taxable"
    - "is_customizable"
    - "is_digital"
    - "file_data"
    - "should_auto_renew"
    - "language"
    - "has_variations"
    - "taxonomy_id"
    - "used_manufacturer"
5. The "tags","materials", and "style" columns contain lists and should be separate tables.

## Clean

In [ ]:
listings_main = df.copy()

#### Change 'Unnamed: 0 ' column to 'main_id'

In [ ]:
listings_main.rename(columns = {'Unnamed: 0':'main_id'}, inplace = True)
listings_main.head(2)

#### Remove non-USD records

In [ ]:
listings_main = listings_main.query('currency_code == "USD"')
listings_main.currency_code.value_counts()

#### Remove irrelevant columns

In [ ]:
drop = ['state','category_id', 'creation_tsz','ending_tsz','original_creation_tsz', 'last_modified_tsz', 
        'sku', 'shop_section_id', 'featured_rank','state_tsz','shipping_template_id',
        'processing_min','processing_max','is_supply','item_weight','item_length','item_width',
        'item_height','item_dimensions_unit','is_private','non_taxable','is_customizable',
        'is_digital','file_data', 'should_auto_renew', 'language', 'has_variations', 'taxonomy_id', 
        'used_manufacturer']

listings_main = listings_main.drop(labels = drop, axis = 1)
listings_main.head(2)

In [ ]:
listings_main.columns

#### Remove "tags","materials", and "style" columns, and save to seperate tables

In [ ]:
# Create intermediate separate tables
listings_tags = listings_main.loc[:,['main_id','tags']]
listings_materials = listings_main.loc[:,['main_id','materials']]
listings_style = listings_main.loc[:,['main_id','style']]

# Explode lists within each row and redefine tables
explode_tags = listings_tags.explode('tags')
listings_tags = pd.DataFrame(explode_tags)

explode_materials = listings_materials.explode('materials')
listings_materials = pd.DataFrame(explode_materials)

explode_style = listings_style.explode('style')
listings_style = pd.DataFrame(explode_style)

# Drop null data
listings_tags = listings_tags.dropna()
listings_materials = listings_materials.dropna()
listings_style = listings_style.dropna()

# Drop columns from main listing table
listings_main = listings_main.drop(labels = ['tags','materials','style'], axis = 1)

In [ ]:
listings_tags.head(3)

In [ ]:
listings_materials.head(3)

In [ ]:
listings_style.head(3)

#### Remove duplicate records

In [ ]:
listings_main = listings_main.drop_duplicates()
listings_main[listings_main.duplicated()]

## Save Clean Datasets

In [ ]:
listings_main.to_csv("/Users/kristinafrazier/documents/projects/etsy/data/nov_2020/listings_main.csv")
listings_materials.to_csv("/Users/kristinafrazier/documents/projects/etsy/data/nov_2020/listings_materials.csv")
listings_tags.to_csv("/Users/kristinafrazier/documents/projects/etsy/data/nov_2020/listings_tags.csv")
listings_style.to_csv("/Users/kristinafrazier/documents/projects/etsy/data/nov_2020/listings_style.csv")